### Taken from http://tensorflow.org/get_started/os_setup.md

Um zu checken, ob die Installation geklappt hat:

In [1]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print sess.run(hello)
a = tf.constant(10)
b = tf.constant(32)
print sess.run(a+b)
sess.close()

Hello, TensorFlow!
42


### Grundlangen

Siehe http://www.tensorflow.org/get_started/basic_usage.md#overview

Mal auf Deutsch übersetzt (vielleicht bringt der Transfer ja was)

Zentrale Datenobjekt ist der Tensor (gleiche wie bei uns der Blob). Die ganzen Berechungen passieren in einem Graphen. Die Knoten des Graphen nennt man `ops`, dort gehen 0 oder mehr Tensoren rein und es kommen 0 oder mehr Tensoren raus. Meist sind diese Ops mathematische Verknüpfungen die passieren (können aber auch Datenquellen sein).

Eine Session startet den Graph (erzeugt wahrscheinlich so was wie eine Instanz).

### Aufbau eines Graphen

Wir fangen mit einem Graphen an, der zwei Konstanten als Quellen hat. Und dann eine Multiplikation macht.

Für die Leute bei denen Python nicht die erste Sprache ist, einen Zeilenvektor und einen Spaltenvector kann man wie folgt erzeugen.

In [2]:
import numpy as np
zv = np.asarray([[3.,3.]])
sp = np.asarray([[2.],[2.]])
zv.shape, sp.shape, np.dot(zv, sp)

((1, 2), (2, 1), array([[ 12.]]))

In [3]:
import tensorflow as tf
# Erster und zweiter Knoten sind Konstanten
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])

product = tf.matmul(matrix1, matrix2) #Ist wieder ein Knoten

### Erzeugen einer Instanz eines Compute graphen
Jetzt erzeugt man so was wie eine Instanz eines Graphen eine Session. Wahrscheinlich ist das sowas wie das compilieren in Theano.

In [4]:
sess = tf.Session() #Starten

# Wie wollen, das Resultat des Produktes haben.
print sess.run(product) #array([[ 12.]], dtype=float32)

# Jetzt wird wahrscheinlich von Product nach oben gegangen und 
# alles gesucht was reinpasst.

print sess.run(matrix1) # macht nicht viel ==> [[3., 3.]]
sess.close() # Sollte man immer machen um Resourcen frei zu machen. 

[[ 12.]]
[[ 3.  3.]]


In [5]:
#Ein python Konstrukt was man sich wahrscheinlich angewöhnen sollte wäre
with tf.Session() as sess:
  print sess.run(product)

[[ 12.]]


### Wo läuft der Graph
TensorFlow kümmert sich darum, wo der Graph läuft (CPU, GPU0, GPU1 usw). Man kann das aber auch kontrolieren:

```
with tf.Session() as sess:
  with tf.device("/gpu:1"):
    matrix1 = tf.constant([[3., 3.]])
    matrix2 = tf.constant([[2.],[2.]])
    product = tf.matmul(matrix1, matrix2)
    ...
```

### Man muss nicht immer kompilieren (Interactive Session)

Mit InteractiveSession und eval kann man auch gleich ausführen.

In [6]:
import tensorflow as tf
sess = tf.InteractiveSession()
print product.eval()

x = tf.Variable(3.0)
x.initializer.run() #Eine Variable muss man noch initialisieren (sonst exception)
print tf.mul(x,product).eval()

sess.close()

[[ 12.]]
[[ 36.]]


### Updating the state of variables

Variablen, sind die Datenstrukturen, welches den state bei verschiedenen Durchläufen des Graphs (sess.run()) beibehalten werden. Zum Beispiel würde man den Gewichte Tensors eines Netzwerkes in einer Variable speichern.

In [7]:
var = tf.Variable(0, name="counter")
one = tf.constant(1)

new_value = tf.add(var, one)
update = tf.assign(var, new_value)
# launched the graph.  We first have to add the `init` Op to the graph.
init_op = tf.initialize_all_variables()

# Launch the graph and run the ops.
with tf.Session() as sess:
    # Run the 'init' op
    sess.run(init_op)
    # Print the initial value of 'var'
    print sess.run(var)
    # Run the op that updates 'var' and print 'var'.
    for _ in range(3):
        print (sess.run(update)) #Gibt den neuen Wert aus
        print sess.run(var) #Gibt den Wert der Veriable aus. Kann auch 0 sein, wenn man ihn vorher nicht upgedated hat

0
1
1
2
2
3
3


### Fetches 
Der Syntax von run ist:
`tf.Session.run(fetches, feed_dict=None)`
Fetches sind die variablen, die man aus dem Graph haben will. Feed_dict sind die Eingaben und Placeholder sind Variablen die erst zur Laufzeit belegt werden. Diese placeholder können dann später die Minibatches sein.

In [8]:
input1 = tf.placeholder(tf.types.float32)
input2 = tf.placeholder(tf.types.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print sess.run([output], feed_dict={input1:[7.], input2:[2.]})

# output:
# [array([ 14.], dtype=float32)]

[array([ 14.], dtype=float32)]
